In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import os

# Define the loss function
criterion = nn.MSELoss()

# Define the compression ratios
compression_ratios = [4,8]

def calculate_mse(outputs, targets):
    mse = criterion(outputs, targets)
    return mse

def NMSE(outputs, inputs_resized):
    outputs_real = torch.reshape(outputs[:, 0, :, :], (outputs.size(0), -1))
    outputs_imag = torch.reshape(outputs[:, 1, :, :], (outputs.size(0), -1))
    outputs_comp = (outputs_real - 0.5) + 1j * (outputs_imag - 0.5)

    inputs_resized_real = torch.reshape(inputs_resized[:, 0, :, :], (inputs_resized.size(0), -1))
    inputs_resized_imag = torch.reshape(inputs_resized[:, 1, :, :], (inputs_resized.size(0), -1))
    inputs_resized_comp = (inputs_resized_real - 0.5) + 1j * (inputs_resized_imag - 0.5)

    mse = torch.mean(torch.abs(outputs_comp - inputs_resized_comp) ** 2, dim=1)
    power = torch.mean(torch.abs(inputs_resized_comp) ** 2, dim=1)

    nmse = 10 * torch.log10(torch.mean(mse / power))

    return nmse


import torch.nn as nn

class EncoderBlock(nn.Module):
    def __init__(self, compressed_dim, height, width):
        super(EncoderBlock, self).__init__()
        self.conv = nn.Conv2d(2, 2, kernel_size=3, stride=1, padding=1)  # Adjusted number of input channels to 4
        self.bn = nn.BatchNorm2d(2)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(2 * height * width, compressed_dim)
        self.height = height
        self.width = width

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.dense(x)
        return x


class DecoderBlock(nn.Module):
    def __init__(self, compressed_dim, height, width):
        super(DecoderBlock, self).__init__()
        self.compressed_dim = compressed_dim
        self.height = height
        self.width = width
        self.dense = nn.Linear(compressed_dim, 2 * (height) * (width))
        self.conv1 = nn.Conv2d(2, 4, kernel_size=1, stride=1)
        self.bn1 = nn.BatchNorm2d(4)
        self.relu1 = nn.ReLU()
        self.conv2_upper = nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1)
        self.bn2_upper = nn.BatchNorm2d(4)
        self.conv2_upper2 = nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=1)
        self.bn2_upper2 = nn.BatchNorm2d(8)
        self.conv2_lower = nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1)
        self.bn2_lower = nn.BatchNorm2d(4)
        self.conv2_lower2 = nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=1)
        self.bn2_lower2 = nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(16, 4, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(4)
        self.conv4 = nn.Conv2d(4, 2, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(2)
        self.reconstruction = nn.Conv2d(2, 2, kernel_size=3, stride=1, padding=1)
        self.sigmoid=nn.Sigmoid()

    def forward(self, x):
        x = self.dense(x)
        x = x.view(-1, 2, self.height, self.width)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)

        upper_branch = self.conv2_upper(x)
        upper_branch = self.bn2_upper(upper_branch)

        upper_branch = self.conv2_upper2(upper_branch)
        upper_branch = self.bn2_upper2(upper_branch)

        lower_branch = self.conv2_lower(x)
        lower_branch = self.bn2_lower(lower_branch)

        lower_branch = self.conv2_lower2(lower_branch)
        lower_branch = self.bn2_lower2(lower_branch)

        x = torch.cat([upper_branch, lower_branch], dim=1)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.reconstruction(x)

        x_reshaped = x.view(-1, 2, self.height, self.width)
        x = x + x_reshaped
        x = self.sigmoid(x)
        return x


class LightweightCNN(nn.Module):
    def __init__(self, compressed_dim, height, width):
        super(LightweightCNN, self).__init__()
        self.encoder = EncoderBlock(compressed_dim, height, width)
        self.decoder = DecoderBlock(compressed_dim, height, width)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


# Define the folder path containing the dataset files
data_folder_path = r"C:\Users\mitra\OneDrive\Desktop\Model4 dataset\Model4 train dataset"
# Get the list of dataset file names in the folder
file_names = sorted(os.listdir(data_folder_path))

# Load the data from all dataset files and combine them
combined_data = []
for file_name in file_names:
    file_path = os.path.join(data_folder_path, file_name)
    data = torch.from_numpy(sio.loadmat(file_path)["x"])
    combined_data.append(data)

combined_data = torch.cat(combined_data, dim=0)

# Shuffle the combined dataset
shuffled_indices = torch.randperm(len(combined_data))
combined_data_shuffled = combined_data[shuffled_indices]

# Create a DataLoader with the shuffled combined dataset
batch_size = 100
shuffled_loader = DataLoader(combined_data_shuffled.float(), batch_size=batch_size, shuffle=True)

# Split the shuffled combined data into train and validation sets
train_data, val_data = train_test_split(combined_data_shuffled, test_size=0.2, random_state=42)

# Normalize the datasets
max_abs_train = torch.max(torch.abs(train_data))
max_abs_val = torch.max(torch.abs(val_data))

train_data_normalized = train_data / max_abs_train * 0.5 + 0.5
val_data_normalized = val_data / max_abs_val * 0.5 + 0.5
# Create a new folder to save model parameters
save_folder = r"C:\Users\mitra\OneDrive\Desktop\Lightweight model parameters for different CRs\All_freq_subband"

for cr in compression_ratios:
    print(f"Training and validating for compression ratio 1/{cr}")

    def plot_train_loss(train_loss_list):
        train_loss = train_loss_list
        plt.plot(range(1, num_epochs + 1), train_loss)
        plt.xlabel('Epoch')
        plt.ylabel('(Training Loss)')
        plt.title('Training Loss vs. Epoch')
        plt.show()

    train_loss_list = []

    # Define the dimensions
    compressed_dim = int(2 * 32 * 32 * (1 / cr))
    height = 32
    width = 32
    model = LightweightCNN(compressed_dim, height, width)
    # Define the optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Create DataLoaders for training and validation data
    train_loader = DataLoader(train_data_normalized.float(), batch_size=100, shuffle=True)
    val_loader = DataLoader(val_data_normalized.float(), batch_size=100,shuffle=True)

    # Train the model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.train()

    num_epochs = 200  # Increase the number of epochs for better training

    for epoch in range(num_epochs):
        running_loss = 0.0

        for inputs in train_loader:
            inputs = inputs.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Resize the input tensor to match the output shape
            inputs_resized = inputs[:, :2, :, :]

            # Compute the MSE loss
            loss = criterion(outputs, inputs_resized)

            # Backward pass
            loss.backward()
            optimizer.step()

            # Update the running loss
            running_loss += loss.item() * inputs.size(0)

        # Calculate the average loss for the training set
        train_loss = running_loss / len(train_loader.dataset)
        train_loss_list.append(train_loss)

        # Validate the model
        model.eval()
        running_loss = 0.0

        with torch.no_grad():
            for val_inputs in val_loader:
                val_inputs = val_inputs.to(device)

                # Forward pass
                val_outputs = model(val_inputs)

                # Resize the input tensor to match the output shape
                val_inputs_resized = val_inputs[:, :2, :, :]

                # Compute the MSE loss
                val_loss = criterion(val_outputs, val_inputs_resized)

                # Update the running loss
                running_loss += val_loss.item() * val_inputs.size(0)

        # Calculate the average loss for the validation set
        val_loss = running_loss / len(val_loader.dataset)

        # Print the epoch and loss for training and validation sets
        print(f"Epoch {epoch + 1}/{num_epochs} - Train Loss: {train_loss} - Val Loss: {val_loss}")
    plot_train_loss(train_loss_list)

    # Save the model/weights
    save_path = os.path.join(save_folder, f"CR_{cr}")
    os.makedirs(save_path, exist_ok=True)
    torch.save(model.state_dict(), os.path.join(save_path, "model_weights.pth"))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import os
import torch.nn.functional as F

# Define the loss function
criterion = nn.MSELoss()

# Define the compression ratios
compression_ratios = [8, 16, 32, 64]

def calculate_mse(outputs, targets):
    mse = criterion(outputs, targets)
    return mse

def NMSE(outputs, inputs_resized):
    outputs_real = torch.reshape(outputs[:, 0, :, :], (outputs.size(0), -1))
    outputs_imag = torch.reshape(outputs[:, 1, :, :], (outputs.size(0), -1))
    outputs_comp = (outputs_real - 0.5) + 1j * (outputs_imag - 0.5)

    inputs_resized_real = torch.reshape(inputs_resized[:, 0, :, :], (inputs_resized.size(0), -1))
    inputs_resized_imag = torch.reshape(inputs_resized[:, 1, :, :], (inputs_resized.size(0), -1))
    inputs_resized_comp = (inputs_resized_real - 0.5) + 1j * (inputs_resized_imag - 0.5)

    mse = torch.mean(torch.abs(outputs_comp - inputs_resized_comp) ** 2, dim=1)
    power = torch.mean(torch.abs(inputs_resized_comp) ** 2, dim=1)

    nmse = 10 * torch.log10(torch.mean(mse / power))

    return nmse


import torch.nn as nn

class EncoderBlock(nn.Module):
    def __init__(self, compressed_dim, height, width):
        super(EncoderBlock, self).__init__()
        self.conv = nn.Conv2d(2, 2, kernel_size=3, stride=1, padding=1)  # Adjusted number of input channels to 4
        self.bn = nn.BatchNorm2d(2)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(2 * height * width, compressed_dim)
        self.height = height
        self.width = width

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.dense(x)
        return x


class DecoderBlock(nn.Module):
    def __init__(self, compressed_dim, height, width):
        super(DecoderBlock, self).__init__()
        self.compressed_dim = compressed_dim
        self.height = height
        self.width = width
        self.dense = nn.Linear(compressed_dim, 2 * (height) * (width))
        self.conv1 = nn.Conv2d(2, 4, kernel_size=1, stride=1)
        self.bn1 = nn.BatchNorm2d(4)
        self.relu1 = nn.ReLU()
        self.conv2_upper = nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1)
        self.bn2_upper = nn.BatchNorm2d(4)
        self.conv2_upper2 = nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=1)
        self.bn2_upper2 = nn.BatchNorm2d(8)
        self.conv2_lower = nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1)
        self.bn2_lower = nn.BatchNorm2d(4)
        self.conv2_lower2 = nn.Conv2d(4, 8, kernel_size=3, stride=1, padding=1)
        self.bn2_lower2 = nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(16, 4, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(4)
        self.conv4 = nn.Conv2d(4, 2, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(2)
        self.reconstruction = nn.Conv2d(2, 2, kernel_size=3, stride=1, padding=1)
        self.sigmoid=nn.Sigmoid()

    def forward(self, x):
        x = self.dense(x)
        x = x.view(-1, 2, self.height, self.width)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)

        upper_branch = self.conv2_upper(x)
        upper_branch = self.bn2_upper(upper_branch)

        upper_branch = self.conv2_upper2(upper_branch)
        upper_branch = self.bn2_upper2(upper_branch)

        lower_branch = self.conv2_lower(x)
        lower_branch = self.bn2_lower(lower_branch)

        lower_branch = self.conv2_lower2(lower_branch)
        lower_branch = self.bn2_lower2(lower_branch)

        x = torch.cat([upper_branch, lower_branch], dim=1)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.reconstruction(x)

        x_reshaped = x.view(-1, 2, self.height, self.width)
        x = x + x_reshaped
        x = self.sigmoid(x)
        return x


class LightweightCNN(nn.Module):
    def __init__(self, compressed_dim, height, width):
        super(LightweightCNN, self).__init__()
        self.encoder = EncoderBlock(compressed_dim, height, width)
        self.decoder = DecoderBlock(compressed_dim, height, width)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


height = 32
width = 32
# Define the folder path containing the test dataset files
test_folder_path = r"C:\Users\mitra\OneDrive\Desktop\Model4 dataset\Model4 test dataset"

# Get the list of test dataset file names in the folder
test_file_names = sorted(os.listdir(test_folder_path))

# Load the test data from all test dataset files and combine them
test_data = []
for test_file_name in test_file_names:
    test_file_path = os.path.join(test_folder_path, test_file_name)
    test_data.append(torch.from_numpy(sio.loadmat(test_file_path)["x"]))

test_data = torch.cat(test_data, dim=0)

# Normalize the test data
max_abs_test = torch.max(torch.abs(test_data))
test_data_normalized = test_data / max_abs_test * 0.5 + 0.5

# Define the function to calculate NMSE
def NMSE(outputs, inputs_resized):
    outputs_real = torch.reshape(outputs[:, 0, :, :], (outputs.size(0), -1))
    outputs_imag = torch.reshape(outputs[:, 1, :, :], (outputs.size(0), -1))
    outputs_comp = (outputs_real - 0.5) + 1j * (outputs_imag - 0.5)

    inputs_resized_real = torch.reshape(inputs_resized[:, 0, :, :], (inputs_resized.size(0), -1))
    inputs_resized_imag = torch.reshape(inputs_resized[:, 1, :, :], (inputs_resized.size(0), -1))
    inputs_resized_comp = (inputs_resized_real - 0.5) + 1j * (inputs_resized_imag - 0.5)

    mse = torch.mean(torch.abs(outputs_comp - inputs_resized_comp) ** 2, dim=1)
    power = torch.mean(torch.abs(inputs_resized_comp) ** 2, dim=1)
    nmse = 10 * torch.log10(torch.mean(mse / power))
    return nmse

# Define the folder paths containing the saved model weights
model_weights_folder = r"C:\Users\mitra\OneDrive\Desktop\Lightweight model parameters for different CRs\All_freq_subband"

model_weights_1_4 = os.path.join(model_weights_folder, "CR_4", "model_weights.pth")
model_weights_1_8 = os.path.join(model_weights_folder, "CR_8", "model_weights.pth")

# Create DataLoader for test data
test_loader = DataLoader(test_data_normalized.float(), batch_size=1, shuffle=False)

# Test the model for each file in the dataset
for i, inputs in enumerate(test_loader, 1):
    inputs = inputs.to(device)

    # Resize the input tensor to match the output shape
    inputs_resized = inputs[:, :2, :, :]

    # Print the file name
    file_name = test_file_names[i - 1]
    print(f"File: {file_name}")

    # Test the model for compression ratio 1/4
    print("Testing for compression ratio 1/4")
    compressed_dim = int(2 * 32 * 32 * (1 / 4))
    model_1_4 = LightweightCNN(compressed_dim, height, width)
    model_1_4.load_state_dict(torch.load(model_weights_1_4))
    model_1_4.to(device)
    model_1_4.eval()

    # Forward pass
    outputs_1_4 = model_1_4(inputs)

    # Compute the NMSE and MSE losses for compression ratio 1/4
    loss_1_4 = NMSE(outputs_1_4, inputs_resized)
    loss_mse_1_4 = F.mse_loss(outputs_1_4, inputs_resized)

    print(f"Compression Ratio 1/4 - NMSE: {loss_1_4.item()} - MSE: {loss_mse_1_4.item()}")

    # Test the model for compression ratio 1/8
    print("Testing for compression ratio 1/8")
    compressed_dim = int(2 * 32 * 32 * (1 / 8))
    model_1_8 = LightweightCNN(compressed_dim, height, width)
    model_1_8.load_state_dict(torch.load(model_weights_1_8))
    model_1_8.to(device)
    model_1_8.eval()

    # Forward pass
    outputs_1_8 = model_1_8(inputs)

    # Compute the NMSE and MSE losses for compression ratio 1/8
    loss_1_8 = NMSE(outputs_1_8, inputs_resized)
    loss_mse_1_8 = F.mse_loss(outputs_1_8, inputs_resized)

    print(f"Compression Ratio 1/8 - NMSE: {loss_1_8.item()} - MSE: {loss_mse_1_8.item()}")
